In [1]:
import pandas as pd
import warnings
import re
import tables
from datetime import datetime
warnings.filterwarnings('ignore')

# Funcao que formata tabela no modelo da genealogia embrapa

In [30]:
def formata_df_no_modelo_genealogia_embrapa(df):
    # Adiciona coluna "id"
    df = df.assign(id=None)
    #Atribui valores de id_produto a id
    df['id'] = df['id_produto']
    # Adiciona coluna "abcz" com valores iguais a null
    df = df.assign(abcz=None)
    #renomeia RGVACA para registro
    df = df.rename(columns={'RGVACA':'registro'})
    #renomeia nomea para nome
    df = df.rename(columns={'nomea':'nome'})
    #renomeia SEXO para sexo
    df = df.rename(columns={'SEXO':'sexo'})
    # Adiciona coluna "paternidade" com valores iguais a null
    df = df.assign(paternidade=None)
    # Adiciona coluna "ck_sire" com valores iguais a null
    df = df.assign(ck_sire=None)
    # Adiciona coluna "ck_dam" com valores iguais a null
    df = df.assign(ck_dam=None)
    # Adiciona coluna "gen_valido" com valores iguais a null
    df = df.assign(gen_valido=None)
    # Adiciona coluna "nasc_est"
    df = df.assign(nasc_est=None)
    #Atribui os valores de nasc a nasc_est
    df['nasc_est'] = df['nasc']
    # Adiciona coluna "gr_psg"
    df = df.assign(gr_psg=None)
    # Adiciona coluna "gr_ps"
    df = df.assign(gr_ps=None)
    # Adiciona coluna "gr_tp"
    df = df.assign(gr_tp=None)
    # Adiciona coluna "rebc2_cri"
    df = df.assign(rebc_cri=None)
    #renomeia rebc_or para rebc_pr
    df = df.rename(columns={'rebc_or':'rebc_pr'})
    # Adiciona coluna "cat"
    if 'cat' in df.columns:
        pass
    else:
        df = df.assign(cat=None)
    # Adiciona coluna "ficha_a"
    df = df.assign(ficha_a=None)
    #renomeia IDANIMAL para idanimal
    df = df.rename(columns={'IDANIMAL':'idanimal'})
    # Adiciona coluna "origem"
    df = df.assign(origem='GIRO')
    # Adiciona coluna "data_insercao"
    df = df.assign(data_insercao=None)
    data_atual = datetime.today().strftime('%Y-%m-%d')
    df['data_insercao'] = data_atual
    df = df[['id_produto', 'id', 'abcz', 'registro','nome','sexo','paternidade','ck_sire','id_touro','id_vaca','ck_dam','gen_valido','nasc','nasc_est','cgen_a','gs','gr_psg','gr_ps','gr_tp','rebc_cri','rebc_pr','cat','ficha_a','cod_prod','idanimal','origem', 'data_insercao']]
    return df


# Lendo planilhas

In [2]:
df_animais_sem_id = pd.read_hdf('df_animais_sem_id.h5', key='df')
df_genealogia_embrapa = pd.read_hdf('df_genealogia_embrapa.h5', key='df')
df_fazendas = pd.read_hdf('df_fazendas.h5', key='df')
df_criadores = pd.read_hdf('df_criadores.h5', key='df')
df_genealogia_abcz = pd.read_hdf('df_genealogia_abcz.h5', key='df')
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')

# Imprimindo dataframes

In [3]:
df_animais_sem_id = df_animais_sem_id.drop_duplicates()
df_animais_sem_id = df_animais_sem_id.drop(columns='id_produto_x')
df_animais_sem_id = df_animais_sem_id.drop(columns='id_produto_y')
#df_animais_sem_id

# Criando dataframe df_animais_sem_id_completo pela juncao do df_animais_sem_id com pedigree
Obs: Mantendo somente o primeiro registro dos animais com IDANIMAL duplicados

In [4]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id, pedigree, on=['IDANIMAL'], how='left', indicator=True, suffixes=('','_drop'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(df_animais_sem_id_completo.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id_completo = df_animais_sem_id_completo[df_animais_sem_id_completo['_merge']=='both']
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates(subset='IDANIMAL')
#df_animais_sem_id_completo

# Criando id_produto dos animais que não possuem id_produto
obs: id_produto = giro+IDANIMAL

In [5]:
df_animais_sem_id_completo['id_produto'] = df_animais_sem_id_completo['id_produto'].fillna('GIRO' + df_animais_sem_id_completo['IDANIMAL'].astype(str))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='_merge')
#df_animais_sem_id_completo

# Juntando df_animais_sem_id_completo (idpai) com genealogia_embrapa (idanimal) tabela separada para pedigree

In [6]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDPAI'], how='left', right_on=['IDANIMAL'], suffixes=('', '_drop'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
#df_animais_sem_id_completo

# Juntando df_animais_sem_id_completo (idmae) com genealogia_embrapa (idanimal)

In [7]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'IDANIMAL']],left_on=['IDMAE'], how='left', right_on=['IDANIMAL'], suffixes=('', '_tes'))
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
#df_animais_sem_id_completo

# Adicionando coluna IDTOURO e id_vaca no df_animais_sem_id_completo

In [8]:
df_animais_sem_id_completo = df_animais_sem_id_completo.rename(columns={'id_produto_drop':'id_touro'})
df_animais_sem_id_completo = df_animais_sem_id_completo.rename(columns={'id_produto_tes':'id_vaca'})
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='IDANIMAL_drop')
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='IDANIMAL_tes')
#df_animais_sem_id_completo

# Buscando o id_touro pelo cod_pai na genealogia_embrapa

In [9]:
df_id_touro = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_pai'], right_on=['cod_prod'], suffixes=('', '_drop'))
#df_id_touro

# Adicionando o id_touro no dataframe com todas as colunas pelo id_animal

In [10]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_id_touro[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
df_animais_sem_id_completo['id_touro'].fillna(df_animais_sem_id_completo['id_produto_drop'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_drop')
#df_animais_sem_id_completo

# Buscando o id_vaca pelo cod_vaca na genealogia_embrapa

In [11]:
df_id_vaca = pd.merge(df_animais_sem_id_completo, df_genealogia_embrapa[['id_produto', 'cod_prod']], left_on=['cod_mae'], right_on=['cod_prod'], indicator=True, suffixes=('', '_drop'))
#df_id_vaca

# Adicionando id_vaca no dataframe com todas as colunas

In [12]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_id_vaca[['IDANIMAL','id_produto_drop']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo = df_animais_sem_id_completo.drop_duplicates()
df_animais_sem_id_completo['id_vaca'].fillna(df_animais_sem_id_completo['id_produto_drop'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_drop')
#df_animais_sem_id_completo

# Pegando o id_produto que vira id_touro dos touros_gir na genealogia embrapa, em que o registro comeca com 3 letras

In [13]:
# somente gs_pai=11 e gs = 0 na genealogia
df_sub_tabela_gir = df_animais_sem_id_completo[df_animais_sem_id_completo['GSPAI']==11]
df_sub_tabela_gir = df_sub_tabela_gir[df_sub_tabela_gir['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#df_sub_tabela_gir

# Pegando o id_produto que vira id_vaca das vacas_gir na genealogia embrapa, em que o registro comeca com 3 letras

In [14]:
# somente gs_pai=11 e gs = 0 na genealogia
df_sub_tabela_gir1 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSMAE']==11]
df_sub_tabela_gir1 = df_sub_tabela_gir1[df_sub_tabela_gir1['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#df_sub_tabela_gir1

# Juntando a genealogia embrapa com os touros comecando com 3 letras e gs 11

In [15]:
df_sub_touros_id = pd.merge(df_sub_tabela_gir, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGPAI'], right_on=['registro'])
#df_sub_touros_id

# Juntando a genealogia embrapa com as vacas comecando com 3 letras e gs 11

In [16]:
df_sub_vacas_id = pd.merge(df_sub_tabela_gir1, df_genealogia_embrapa[['id_produto','registro']], left_on=['RGMAE'], right_on=['registro'])
#df_sub_vacas_id

In [17]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_sub_touros_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_touro'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, df_sub_vacas_id[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_vaca'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
#df_animais_sem_id_completo

Juntar codigo acima com genealogia
RGPAI(df_sub_tabela_gir) liga com registro(genealogia)
duplicar solucao para mae
RGPAI RGMAE RGVACA GS=11 retirar acentos e espacos

# Pegando os id_produto's dos touros_gir na genealogia embrapa, em que o registro nao comeca com 3 letras

In [18]:
df_sub_tabela_gir1 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSPAI']==11]
df_sub_tabela_gir1 = df_sub_tabela_gir1[~df_sub_tabela_gir1['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#Pegando os 4 primeiros caracteres do nome
df_sub_tabela_gir1['nome4caracteres'] = df_sub_tabela_gir1['nomep'].str.slice(start=0, stop=4)


df_subtabela_genealogia = df_genealogia_embrapa
df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['gs']==0]
df_subtabela_genealogia = df_subtabela_genealogia[df_subtabela_genealogia['sexo']=='M']
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].isnull()]
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['nome'].isnull()]
df_subtabela_genealogia = df_subtabela_genealogia[~df_subtabela_genealogia['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
df_subtabela_genealogia['nome4caracteres'] = df_subtabela_genealogia['nome'].str.slice(start=0, stop=4)

dj_juntando_tabelas = pd.merge(df_sub_tabela_gir1, df_subtabela_genealogia[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGPAI'], right_on=['nome4caracteres', 'registro'])
#dj_juntando_tabelas

# Pegando os id_produto’s das vacas na genealogia embrapa, em que o registro nao comeca com 3 letras

In [19]:
df_sub_tabela_gir2 = df_animais_sem_id_completo[df_animais_sem_id_completo['GSMAE']==11]
df_sub_tabela_gir2 = df_sub_tabela_gir2[~df_sub_tabela_gir2['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
#Pegando os 4 primeiros caracteres do nome
df_sub_tabela_gir2['nome4caracteres'] = df_sub_tabela_gir2['nomem'].str.slice(start=0, stop=4)


df_subtabela_genealogia1 = df_genealogia_embrapa
df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['gs']==0]
df_subtabela_genealogia1 = df_subtabela_genealogia1[df_subtabela_genealogia1['sexo']=='F']
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].isnull()]
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['nome'].isnull()]
df_subtabela_genealogia1 = df_subtabela_genealogia1[~df_subtabela_genealogia1['registro'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]
df_subtabela_genealogia1['nome4caracteres'] = df_subtabela_genealogia1['nome'].str.slice(start=0, stop=4)

dj_juntando_tabelas1 = pd.merge(df_sub_tabela_gir2, df_subtabela_genealogia1[['id_produto', 'registro', 'nome4caracteres']], left_on=['nome4caracteres', 'RGMAE'], right_on=['nome4caracteres', 'registro'])
#dj_juntando_tabelas1

# Adicionando ID's touros e ID's vacas encontrados em que o registro não comeca com 3 letras na df_animais_sem_id_completo

In [20]:
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, dj_juntando_tabelas[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_touro'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
df_animais_sem_id_completo = pd.merge(df_animais_sem_id_completo, dj_juntando_tabelas1[['IDANIMAL', 'id_produto_y']], on=['IDANIMAL'], how='left')
df_animais_sem_id_completo['id_vaca'].fillna(df_animais_sem_id_completo['id_produto_y'], inplace=True)
df_animais_sem_id_completo = df_animais_sem_id_completo.drop(columns='id_produto_y')
#df_animais_sem_id_completo

# Lista de animais com id_vaca E id_touro

In [21]:
df_animais_com_ids = df_animais_sem_id_completo[~df_animais_sem_id_completo['id_vaca'].isnull() & ~df_animais_sem_id_completo['id_touro'].isnull()]
#df_animais_com_ids

# Lista de animais sem id_touro e sem id_vaca

In [22]:
df_animais_sem_nenhum_ids = df_animais_sem_id_completo[df_animais_sem_id_completo['id_vaca'].isnull() & df_animais_sem_id_completo['id_touro'].isnull()]
df_animais_sem_nenhum_ids = df_animais_sem_nenhum_ids.drop_duplicates()
df_animais_sem_nenhum_ids

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
14,8494,WILY 1151 DA JCS C.BONITA,38,3468-BJ,0,0,0,0,0,0,...,2019,WILY 1151 DA JCS C.BONITA,04/02/2019,F,NaN,NaN,3/8HOL 5/8GIR,417,NaN,NaN
35,5192,LIGIA FIV HUMBLENKIND 2403 RS,34,0578-BG,COOKIECUTTER HUMBLENKIND-ET,10,AX152772,KIRIA FIV ALAMBARI,12,5885-AT,...,2019,LIGIA FIV HUMBLENKIND 2403 RS DO RANCHO ALEGRE,30/04/2019,F,COOKIECUTTER HUMBLENKIND-ET,KIRIA FIV ALAMBARI,3/4HOL 1/4GIR,795,NaN,NaN
38,8494,VAIDOSA 1108 DA JCS C.BONITA,14,3470-BJ,0,0,0,0,0,0,...,2018,VAIDOSA 1108 DA JCS C.BONITA,18/08/2018,F,NaN,NaN,1/4HOL 3/4GIR,315,NaN,NaN
54,10032,ELITE RECANTO DO COQUEIRO,58,8014-BH,0,0,0,0,0,0,...,2019,ELITE RECANTO DO COQUEIRO,25/08/2019,F,NaN,NaN,5/8HOL 3/8GIR,693,NaN,NaN
65,10032,BELEZA RECANTO DO COQUEIRO,12,3748-AS,0,0,0,0,0,0,...,2014,BELEZA RECANTO DO COQUEIRO,10/09/2014,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,1026,ARTEMIS FBS CAETANO,12,6396-AR,0,0,0,0,0,0,...,2013,ARTEMIS FBS CAETANO,22/10/2013,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
528,5191,ORGANICA 5/8-591 2K,34,3860-BD,CACIQUE A. JORDAN NOVA TERRA,58,9140-M,GRINALDA 2K,78,6223-AA,...,2018,ORGANICA 5/8-591 2K,03/04/2018,F,CACIQUE A. JORDAN NOVA TERRA,GRINALDA 2K,3/4HOL 1/4GIR,795,NaN,NaN
530,10032,SERRA NEGRA DA SANQUIT,12,5043-AP,0,0,0,0,0,0,...,2013,SERRA NEGRA DA SANQUIT,31/05/2013,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
542,1129,UARA ANGICO,34,8692-AX,0,0,0,0,0,0,...,2017,UARA ANGICO,03/02/2017,F,NaN,NaN,3/4HOL 1/4GIR,795,NaN,NaN


# Lista de animais sem id_vaca

In [40]:
df_animais_sem_id_vaca = df_animais_sem_id_completo[df_animais_sem_id_completo['id_vaca'].isnull()]
df_animais_sem_id_vaca = df_animais_sem_id_vaca.drop_duplicates()
df_animais_sem_id_vaca

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2015,NANA IMPERADOR BAXTER RF SAO FRANCISCO,11/03/2015,F,IMPERADOR BAXTER VOLTA FRIA,JURITI HERBY RF SAO FRANCISCO,5/8HOL 3/8GIR,666,GIRO977401,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2013,EDILAINE IMPERIO FIV 4303 JARDIM DO TURVO,18/10/2013,F,IMPERIO TE STA EDWIGES,SERTEC QATAR FABIANA PROGRESS,1/2HOL 1/2GIR,519,RIG163,NaN
5,10586,JHV CANASTRA ENLEVO,12,5011-BG,ENLEVO SILVANIA,11,EFC717,JHV FORMATURA 315,10,C-3602,...,2019,JHV CANASTRA ENLEVO,18/06/2019,F,ENLEVO SILVANIA,JHV FORMATURA 315,1/2HOL 1/2GIR,519,EFC717,NaN
8,22541,KELLY ESPELHO JF DA SANTA RITA,12,7974-AM,ESPELHO TE DE BRASILIA,11,RRP5664,RACKELY JF DA SANTA RITA,10,A-7744,...,2012,KELLY ESPELHO JF DA SANTA RITA,06/04/2012,F,ESPELHO TE DE BRASILIA,RACKELY JF DA SANTA RITA,1/2HOL 1/2GIR,519,RRP5664,NaN
10,4406,VITORIA WINDSTAR IRARA II,34,6762-B,DUPASQUIER WINDSTAR,10,AX101932,SANDI IRARA II,12,X-2396,...,2009,VITORIA WINDSTAR IRARA II,18/05/2009,F,DUPASQUIER WINDSTAR,SANDI IRARA II,3/4HOL 1/4GIR,795,143607,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,9330,PALMADA CAL,14,0383-BN,DIORITO CAL,11,CAL9630,ILHOA FIV CAL,12,7229-AG,...,2020,PALMADA CAL,22/02/2020,F,DIORITO CAL,ILHOA FIV CAL,1/4HOL 3/4GIR,315,CAL9630,NaN
548,6029,GRAVINA FEBO FR RECREIO,5858,9073-AS,FEBO FR RECREIO,58,6300-N,DORINA FR RECREIO,58,0860-T,...,2016,GRAVINA FEBO FR RECREIO,13/06/2016,F,FEBO FR RECREIO,DORINA FR RECREIO,5/8HOL 3/8GIR,666,GIRO1114289,NaN
553,10032,CATUABA F.DA BARRA,12,2720-BM,0,0,0,0,0,0,...,2015,CATUABA F.DA BARRA,01/09/2015,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
557,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2019,LIA DAS ARABIAS,13/06/2019,F,TURBO DAS ARABIAS,LILIKA DAS ARABIAS,5/8HOL 3/8GIR,666,GIRO1155542,NaN


# Lista de animais sem id_touro

In [24]:
df_animais_sem_id_touro = df_animais_sem_id_completo[df_animais_sem_id_completo['id_touro'].isnull()]
df_animais_sem_id_touro = df_animais_sem_id_touro.drop_duplicates()
df_animais_sem_id_touro

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
9,10586,MORANGA OPINIOSO DO SEGREDO VE,58,6930-BC,ARATINGA OPINIOSO 81 BROKAW TE,10,AX144696,SALINA NOBRE DO MORRO,14,C-4972,...,2018,MORANGA OPINIOSO DO SEGREDO VELHO,07/08/2018,F,ARATINGA OPINIOSO 81 BROKAW TE,SALINA NOBRE DO MORRO,5/8HOL 3/8GIR,693,NaN,GIRO821357
14,8494,WILY 1151 DA JCS C.BONITA,38,3468-BJ,0,0,0,0,0,0,...,2019,WILY 1151 DA JCS C.BONITA,04/02/2019,F,NaN,NaN,3/8HOL 5/8GIR,417,NaN,NaN
15,4237,JADI DYNAMO FIV FUNDAO,12,3640-BK,MR RUBICON DYNAMO-ET,10,AX152161,DIAMANTINA FIV ZBR,11,ZBR157,...,2019,JADI DYNAMO FIV FUNDAO,14/09/2019,F,MR RUBICON DYNAMO-ET,DIAMANTINA FIV ZBR,1/2HOL 1/2GIR,519,NaN,ZBR157
16,5569,CLAIRE UTOPIA LAMBDA SH FIV 2B,12,9276-BK,FARNEAR DELTA-LAMBDA-ET,10,AX152152,UTOPIA FIV 2B,11,ZAB1116,...,2019,CLAIRE UTOPIA LAMBDA SH FIV 2B,02/10/2019,F,FARNEAR DELTA-LAMBDA-ET,UTOPIA FIV 2B,1/2HOL 1/2GIR,519,NaN,ZAB1116
21,52717,TANISHA FIV MORENO ALEGRE,58,4758-BB,PEAK ALTAMORENO-ET,10,AX150663,PRATEADA FIV SANSAO JGVA,14,3599-F,...,2018,TANISHA FIV MORENO ALEGRE,19/03/2018,F,PEAK ALTAMORENO-ET,PRATEADA FIV SANSAO JGVA,5/8HOL 3/8GIR,693,NaN,GIRO984858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,5191,ORGANICA 5/8-591 2K,34,3860-BD,CACIQUE A. JORDAN NOVA TERRA,58,9140-M,GRINALDA 2K,78,6223-AA,...,2018,ORGANICA 5/8-591 2K,03/04/2018,F,CACIQUE A. JORDAN NOVA TERRA,GRINALDA 2K,3/4HOL 1/4GIR,795,NaN,NaN
530,10032,SERRA NEGRA DA SANQUIT,12,5043-AP,0,0,0,0,0,0,...,2013,SERRA NEGRA DA SANQUIT,31/05/2013,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN
542,1129,UARA ANGICO,34,8692-AX,0,0,0,0,0,0,...,2017,UARA ANGICO,03/02/2017,F,NaN,NaN,3/4HOL 1/4GIR,795,NaN,NaN
553,10032,CATUABA F.DA BARRA,12,2720-BM,0,0,0,0,0,0,...,2015,CATUABA F.DA BARRA,01/09/2015,F,NaN,NaN,1/2HOL 1/2GIR,519,NaN,NaN


# Separando touros dos animais que nao possuem id_touro
Somente se IDPAI != 0

In [44]:
df_animais_sem_id_touro_gir = df_animais_sem_id_touro[df_animais_sem_id_touro['GSPAI']==0]
df_animais_sem_id_touro_gir = df_animais_sem_id_touro_gir.drop_duplicates()
df_animais_sem_id_touro_gir

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca


# IDPAI != Nan

Lista abaixo juntar ID_PAI com id_animal no pedigree
drop duplicate id_animal
criar id_touro nessa tabela sendo GIRO+IDPAI (quando idPAI != nan)
id_touro vai virar id_produto na df_animais_com_ids
id_pai e id_mae nulo pausa a recursividade

In [43]:
df_animais_sem_id_touro = df_animais_sem_id_touro[df_animais_sem_id_touro['GSPAI']!=0]
df_animais_sem_id_touro = df_animais_sem_id_touro.drop_duplicates()
df_animais_sem_id_touro

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,AN,NOMEVACA,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca
9,10586,MORANGA OPINIOSO DO SEGREDO VE,58,6930-BC,ARATINGA OPINIOSO 81 BROKAW TE,10,AX144696,SALINA NOBRE DO MORRO,14,C-4972,...,2018,MORANGA OPINIOSO DO SEGREDO VELHO,07/08/2018,F,ARATINGA OPINIOSO 81 BROKAW TE,SALINA NOBRE DO MORRO,5/8HOL 3/8GIR,693,NaN,GIRO821357
15,4237,JADI DYNAMO FIV FUNDAO,12,3640-BK,MR RUBICON DYNAMO-ET,10,AX152161,DIAMANTINA FIV ZBR,11,ZBR157,...,2019,JADI DYNAMO FIV FUNDAO,14/09/2019,F,MR RUBICON DYNAMO-ET,DIAMANTINA FIV ZBR,1/2HOL 1/2GIR,519,NaN,ZBR157
16,5569,CLAIRE UTOPIA LAMBDA SH FIV 2B,12,9276-BK,FARNEAR DELTA-LAMBDA-ET,10,AX152152,UTOPIA FIV 2B,11,ZAB1116,...,2019,CLAIRE UTOPIA LAMBDA SH FIV 2B,02/10/2019,F,FARNEAR DELTA-LAMBDA-ET,UTOPIA FIV 2B,1/2HOL 1/2GIR,519,NaN,ZAB1116
21,52717,TANISHA FIV MORENO ALEGRE,58,4758-BB,PEAK ALTAMORENO-ET,10,AX150663,PRATEADA FIV SANSAO JGVA,14,3599-F,...,2018,TANISHA FIV MORENO ALEGRE,19/03/2018,F,PEAK ALTAMORENO-ET,PRATEADA FIV SANSAO JGVA,5/8HOL 3/8GIR,693,NaN,GIRO984858
25,2381,FAUNA LEMUST JM NOVO HORIZONTE,34,9247-AV,COMESTAR LEMUST,10,AX150051,BERLINDA JAYVEN JM NOVO HORIZO,12,5203-AE,...,2017,FAUNA LEMUST JM NOVO HORIZONTE,01/07/2017,F,COMESTAR LEMUST,BERLINDA JAYVEN JM NOVO HORIZO,3/4HOL 1/4GIR,795,NaN,GIRO1388188
26,7698,HERMIDA TWISTER LPN,12,8457-AX,PRIDE MOGUL TWISTER CRI-ET,10,AX146367,FANTASIA FIV,11,CAJG1,...,2017,HERMIDA TWISTER LPN,12/03/2017,F,PRIDE MOGUL TWISTER CRI-ET,FANTASIA FIV,1/2HOL 1/2GIR,519,NaN,CAJG1
35,5192,LIGIA FIV HUMBLENKIND 2403 RS,34,0578-BG,COOKIECUTTER HUMBLENKIND-ET,10,AX152772,KIRIA FIV ALAMBARI,12,5885-AT,...,2019,LIGIA FIV HUMBLENKIND 2403 RS DO RANCHO ALEGRE,30/04/2019,F,COOKIECUTTER HUMBLENKIND-ET,KIRIA FIV ALAMBARI,3/4HOL 1/4GIR,795,NaN,NaN
48,3304,FAZENDEIRA DA 4 RS,12,3379-AR,DEANGATE QUENTIN,10,AX136310,GILES VILLEFORT,11,IVAR3186,...,2015,FAZENDEIRA DA 4 RS,08/09/2015,F,DEANGATE QUENTIN,GILES VILLEFORT,1/2HOL 1/2GIR,519,NaN,IVAR3186
68,6039,FECULA BRASIL COC,12,4846-AS,POITARA BRASIL GOLDWYN-TE,10,AX138212,UXA CAL,11,CAL7085,...,2016,FECULA BRASIL COC,27/07/2016,F,POITARA BRASIL GOLDWYN-TE,UXA CAL,1/2HOL 1/2GIR,519,NaN,CAL7085
74,5192,LAKINA TROY 2516 RS DO RANCHO,34,1676-BK,RIVER-BRIDGE CO-OP TROY-ET,10,AX144153,KEITY JACKSON FIV CRAVINA 2252,12,3149-AZ,...,2019,LAKINA TROY 2516 RS DO RANCHO ALEGRE,12/12/2019,F,RIVER-BRIDGE CO-OP TROY-ET,KEITY JACKSON FIV CRAVINA 2252,3/4HOL 1/4GIR,795,NaN,NaN


# Juntar df_animais_sem_id_touro pelo id_pai com id_animal no pedigree

In [42]:
df_teste = pd.merge(df_animais_sem_id_touro, pedigree, left_on=['IDPAI'], right_on=['IDANIMAL'], suffixes=('', '_drop'))
df_teste = df_teste.drop_duplicates()
df_teste = df_teste.drop_duplicates(subset ='IDANIMAL')
df_teste

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,RGPAI_drop,GSPAI_drop,IDPAI_drop,NOMEMAE_drop,RGMAE_drop,GSMAE_drop,IDMAE_drop,gs_drop,cgen_a_drop,comb_drop
0,10586,MORANGA OPINIOSO DO SEGREDO VE,58,6930-BC,ARATINGA OPINIOSO 81 BROKAW TE,10,AX144696,SALINA NOBRE DO MORRO,14,C-4972,...,AX137993,10.0,1423735.0,ARATINGA OPINIOSA 505 JAYZ,BX405461,10.0,1654542.0,8,HOL,999
1,4237,JADI DYNAMO FIV FUNDAO,12,3640-BK,MR RUBICON DYNAMO-ET,10,AX152161,DIAMANTINA FIV ZBR,11,ZBR157,...,AX145171,10.0,1841840.0,MISS OCD ROBST DELICIOUS-ET,8403006989479,10.0,2942706.0,8,HOL,999
2,5569,CLAIRE UTOPIA LAMBDA SH FIV 2B,12,9276-BK,FARNEAR DELTA-LAMBDA-ET,10,AX152152,UTOPIA FIV 2B,11,ZAB1116,...,AX145175,10.0,1816215.0,EDG LORETTE UNO 2198-ET,US72437745,10.0,2192738.0,8,HOL,999
3,52717,TANISHA FIV MORENO ALEGRE,58,4758-BB,PEAK ALTAMORENO-ET,10,AX150663,PRATEADA FIV SANSAO JGVA,14,3599-F,...,AX144190,10.0,1961534.0,OCD MOGUL ABRACADABRA-ET,8403009533582,10.0,1703940.0,8,HOL,999
4,2381,FAUNA LEMUST JM NOVO HORIZONTE,34,9247-AV,COMESTAR LEMUST,10,AX150051,BERLINDA JAYVEN JM NOVO HORIZO,12,5203-AE,...,AX125509,10.0,981679.0,COMESTAR LAUTAMIA BOLTON,CHFHB-104226402,10.0,1361152.0,8,HOL,999
5,7698,HERMIDA TWISTER LPN,12,8457-AX,PRIDE MOGUL TWISTER CRI-ET,10,AX146367,FANTASIA FIV,11,CAJG1,...,AX138953,10.0,1299782.0,PRIDE FREDDIE TATA 905-ET,US69491314,10.0,1973856.0,8,HOL,999
6,5192,LIGIA FIV HUMBLENKIND 2403 RS,34,0578-BG,COOKIECUTTER HUMBLENKIND-ET,10,AX152772,KIRIA FIV ALAMBARI,12,5885-AT,...,AX149199,10.0,1794846.0,COOKIECUTTER EPIC HAZEL-ET,8403009848497,10.0,2053184.0,8,HOL,999
7,3304,FAZENDEIRA DA 4 RS,12,3379-AR,DEANGATE QUENTIN,10,AX136310,GILES VILLEFORT,11,IVAR3186,...,225243774,10.0,1478390.0,DEANGATE QUEENIE 14,9642509,10.0,1478391.0,8,HOL,999
8,6039,FECULA BRASIL COC,12,4846-AS,POITARA BRASIL GOLDWYN-TE,10,AX138212,UXA CAL,11,CAL7085,...,AX118004,10.0,912507.0,ASTRAHOE RBRT DPLX RAYNA-ET,HFHB-137525340,10.0,1374731.0,8,HOL,999
9,6039,FACA BRASIL COC,12,8550-AM,POITARA BRASIL GOLDWYN-TE,10,AX138212,EVA FIV,11,JCVI27,...,AX118004,10.0,912507.0,ASTRAHOE RBRT DPLX RAYNA-ET,HFHB-137525340,10.0,1374731.0,8,HOL,999


Tabela de fazendas pegar o rebc2_pr -> rebc2-or (fazendas) <- reb_giro(df_animais_com_id)

ABCZ - S = Quando cod prod > 0 ABCZ
cat cod_prod >0 (5 comprac 11) ABCZ

id = id_produto
nas e nasc_est = DNASC


# Separando os animais com id_touro e id_vaca que possuem COMPRAC = 11 dos que nao possuem

In [27]:
# Pegando rebc2_pr da tabela de fazendas
df_animais_com_ids = pd.merge(df_animais_com_ids, df_criadores[['rebc_or', 'codgiro']], left_on=['reb_giro'], right_on=['codgiro'])
df_animais_com_ids = df_animais_com_ids.drop_duplicates()
df_animais_com_ids_abcz = df_animais_com_ids[df_animais_com_ids['COMPRAC']==11]
df_animais_com_ids = df_animais_com_ids[df_animais_com_ids['COMPRAC']!=11]
df_animais_com_ids

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca,rebc_or,codgiro
0,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,22/12/2014,F,MS ATLEES SHT AFTERSHOCK - ET,JOANE FIV KUBERA,1/2HOL 1/2GIR,519,207142,ACFG1840,10639-GIRO,10639
1,10639,FLORENTINA FIV ATWOOD SAMPA,12,0949-AK,MAPLE-DOWNS-I G W ATWOOD-ET,10,AX130581,JULIA FIV KUBERA,11,ACFG1848,...,05/08/2014,F,MAPLE-DOWNS-I G W ATWOOD-ET,JULIA FIV KUBERA,1/2HOL 1/2GIR,519,208574,ACFG1848,10639-GIRO,10639
2,10639,JGSA 1216 FIV GILCREST PORTEIR,12,9419-BI,DE-SU ALTAGILCREST-ET,10,AX141066,HERDEIRA,11,ERD137,...,08/08/2018,F,DE-SU ALTAGILCREST-ET,HERDEIRA,1/2HOL 1/2GIR,519,GIRO1455287,ERD137,10639-GIRO,10639
3,10639,CANJA FIV FINEST PORTEIRA AZUL,12,5893-AU,PENN-GATE FINEST-ET,10,AX117712,VALSA DA CINEL,11,LVRG574,...,31/01/2014,F,PENN-GATE FINEST-ET,VALSA DA CINEL,1/2HOL 1/2GIR,519,GIRO812244,LVRG574,10639-GIRO,10639
4,10639,BEATRIZ TEATRO DO MEU SONHO,12,5941-AU,TEATRO DA SILVANIA,11,EFC383,FABULA DO MEU SONHO,10,C-3722,...,12/05/2016,F,TEATRO DA SILVANIA,FABULA DO MEU SONHO,1/2HOL 1/2GIR,519,EFC383,GIRO1845621,10639-GIRO,10639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,8030,SAVANA PLANET FIV FZD,12,4961-U,ENSENADA TABOO PLANET-ET,10,AX125509,NINA JAGUAR,11,FAZP84,...,31/07/2013,F,ENSENADA TABOO PLANET-ET,NINA JAGUAR,1/2HOL 1/2GIR,519,212019,FAZP84,26363-ABCZ,8030
300,8030,SAVANA PLANET FIV FZD,12,4961-U,ENSENADA TABOO PLANET-ET,10,AX125509,NINA JAGUAR,11,FAZP84,...,31/07/2013,F,ENSENADA TABOO PLANET-ET,NINA JAGUAR,1/2HOL 1/2GIR,519,212019,FAZP84,4655-ABCZ,8030
301,2389,TEQUILA GENGIS KHAN FIV CAMPAN,12,5278-AZ,GENGIS KHAN DE BRASILIA,11,RRP6097,RHOELANDT 384 BETSY SPIRIT ROY,10,BX323094,...,16/02/2017,F,GENGIS KHAN DE BRASILIA,RHOELANDT 384 BETSY SPIRIT ROY,1/2HOL 1/2GIR,519,RRP6097,GIRO1054443,2389-GIRO,2389
302,2384,BATUCADA 5003 EXPANDER FIV JM,34,7184-BC,STANTONS EXPANDER-ET,10,AX149579,COROLA JM MONTE ALVERNE,12,AA-7520,...,01/12/2017,F,STANTONS EXPANDER-ET,COROLA JM MONTE ALVERNE,3/4HOL 1/4GIR,795,GIRO1857624,GIRO793074,2384-GIRO,2384


# Tabela com os animais que possuem comprac = 11

In [28]:
df_animais_com_ids_abcz

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,DNASC,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca,rebc_or,codgiro
6,1945,BROMELIA FIV DO JRD,11,JRDG28,RADAR DOS POCOES,11,A7368,PROFANA 6,11,EBQG1,...,01/01/2011,F,RADAR DOS POCOES,PROFANA 6,GIR,11,103136,EBQG1,37634-ABCZ,1945
9,1945,CATITA VITORIA DE OG,11,OGM735,TEATRO DA SILVANIA,11,EFC383,BRASILEIRA,11,JOR4134,...,10/01/2015,F,TEATRO DA SILVANIA,BRASILEIRA,GIR,11,EFC383,JOR4134,37634-ABCZ,1945
15,1945,BAILARINA,11,MBLO12,GENGIS KHAN DE BRASILIA,11,RRP6097,BEL DA RTPA,11,NLT89,...,02/01/2020,F,GENGIS KHAN DE BRASILIA,BEL DA RTPA,GIR,11,RRP6097,NLT89,37634-ABCZ,1945
17,1945,FANTASIA,11,MBLO18,JAGUAR TE DO GAVIAO,11,GAV291,ELECTRA FIV PEDREIRA,11,PPG41,...,02/01/2020,F,JAGUAR TE DO GAVIAO,ELECTRA FIV PEDREIRA,GIR,11,GAV291,PPG41,37634-ABCZ,1945
18,1945,PANTOJA VILLEFORT,11,IVAR5160,TABU TE DA CAL,11,CAL6557,CACULA VILLEFORT,11,IVAR561,...,16/12/2015,F,TABU TE DA CAL,CACULA VILLEFORT,GIR,11,CAL6557,IVAR561,37634-ABCZ,1945
146,5191,IBERICA FARDO,11,APKK18,FARDO FIV F.MUTUM,11,MUT697,ESPANHOLA,11,APKK1,...,27/09/2013,F,FARDO FIV F.MUTUM,ESPANHOLA,GIR,11,MUT697,APKK1,27111-ABCZ,5191
184,5678,FIV 5035,11,GBEZ7,JAGUAR TE DO GAVIAO,11,GAV291,ABRIGADA TCP,11,TPMG25,...,24/08/2015,F,JAGUAR TE DO GAVIAO,ABRIGADA TCP,GIR,11,GAV291,TPMG25,5678-GIRO,5678
241,5407,GARFIEIRA,11,PVBG1140,RADAR DOS POCOES,11,A7368,LARA FIV DA SADONANA,11,SDNA34,...,30/09/2013,F,RADAR DOS POCOES,LARA FIV DA SADONANA,GIR,11,103136,SDNA34,5407-GIRO,5407
244,5407,GL CERINA CRISTAL,11,PVBG1091,JAGUAR TE DO GAVIAO,11,GAV291,JULY FIV KUBERA,11,ACFG1825,...,19/08/2013,F,JAGUAR TE DO GAVIAO,JULY FIV KUBERA,GIR,11,GAV291,ACFG1825,5407-GIRO,5407
245,5407,FORTUNA DA CRISTAL,11,PVBG855,RADAR DOS POCOES,11,A7368,RADULA TE DO GAVIAO,11,GAV980,...,18/12/2012,F,RADAR DOS POCOES,RADULA TE DO GAVIAO,GIR,11,103136,GAV980,5407-GIRO,5407


# Juntando a tabela dos animais com ids encontrados que possuem comprac = 11 com o genealogia abcz

In [29]:
df_animais_com_ids_abcz = pd.merge(df_animais_com_ids_abcz, df_genealogia_abcz[['cod_prod', 'cat']], on=['cod_prod'])
df_animais_com_ids_abcz

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,SEXO,NOMEPAI,NOMEMAE,cgen_a,comb,id_touro,id_vaca,rebc_or,codgiro,cat
0,1945,BROMELIA FIV DO JRD,11,JRDG28,RADAR DOS POCOES,11,A7368,PROFANA 6,11,EBQG1,...,F,RADAR DOS POCOES,PROFANA 6,GIR,11,103136,EBQG1,37634-ABCZ,1945,PO
1,1945,CATITA VITORIA DE OG,11,OGM735,TEATRO DA SILVANIA,11,EFC383,BRASILEIRA,11,JOR4134,...,F,TEATRO DA SILVANIA,BRASILEIRA,GIR,11,EFC383,JOR4134,37634-ABCZ,1945,PO
2,1945,BAILARINA,11,MBLO12,GENGIS KHAN DE BRASILIA,11,RRP6097,BEL DA RTPA,11,NLT89,...,F,GENGIS KHAN DE BRASILIA,BEL DA RTPA,GIR,11,RRP6097,NLT89,37634-ABCZ,1945,PO
3,1945,FANTASIA,11,MBLO18,JAGUAR TE DO GAVIAO,11,GAV291,ELECTRA FIV PEDREIRA,11,PPG41,...,F,JAGUAR TE DO GAVIAO,ELECTRA FIV PEDREIRA,GIR,11,GAV291,PPG41,37634-ABCZ,1945,PO
4,1945,PANTOJA VILLEFORT,11,IVAR5160,TABU TE DA CAL,11,CAL6557,CACULA VILLEFORT,11,IVAR561,...,F,TABU TE DA CAL,CACULA VILLEFORT,GIR,11,CAL6557,IVAR561,37634-ABCZ,1945,PO
5,5191,IBERICA FARDO,11,APKK18,FARDO FIV F.MUTUM,11,MUT697,ESPANHOLA,11,APKK1,...,F,FARDO FIV F.MUTUM,ESPANHOLA,GIR,11,MUT697,APKK1,27111-ABCZ,5191,PO
6,5678,FIV 5035,11,GBEZ7,JAGUAR TE DO GAVIAO,11,GAV291,ABRIGADA TCP,11,TPMG25,...,F,JAGUAR TE DO GAVIAO,ABRIGADA TCP,GIR,11,GAV291,TPMG25,5678-GIRO,5678,PO
7,5407,GARFIEIRA,11,PVBG1140,RADAR DOS POCOES,11,A7368,LARA FIV DA SADONANA,11,SDNA34,...,F,RADAR DOS POCOES,LARA FIV DA SADONANA,GIR,11,103136,SDNA34,5407-GIRO,5407,PO
8,5407,GL CERINA CRISTAL,11,PVBG1091,JAGUAR TE DO GAVIAO,11,GAV291,JULY FIV KUBERA,11,ACFG1825,...,F,JAGUAR TE DO GAVIAO,JULY FIV KUBERA,GIR,11,GAV291,ACFG1825,5407-GIRO,5407,PO
9,5407,FORTUNA DA CRISTAL,11,PVBG855,RADAR DOS POCOES,11,A7368,RADULA TE DO GAVIAO,11,GAV980,...,F,RADAR DOS POCOES,RADULA TE DO GAVIAO,GIR,11,103136,GAV980,5407-GIRO,5407,PO


# Formatando dataframes no modelo da tabela Genealogia Embrapa do Banco de dados

In [31]:
df_animais_com_ids_abcz = formata_df_no_modelo_genealogia_embrapa(df_animais_com_ids_abcz)
df_animais_com_ids = formata_df_no_modelo_genealogia_embrapa(df_animais_com_ids)
df_animais_com_ids = pd.concat([df_animais_com_ids, df_animais_com_ids_abcz])
df_animais_com_ids

,id_produto,id,abcz,registro,nome,sexo,sexo,paternidade,ck_sire,id_touro,...,gr_ps,gr_tp,rebc_cri,rebc_pr,cat,ficha_a,cod_prod,idanimal,origem,data_insercao
0,GIRO1413366,GIRO1413366,None,JRDG28,BROMELIA FIV DO JRD,F,F,None,None,103136,...,None,None,None,37634-ABCZ,PO,None,12005290.0,1413366,ABCZ,2023-03-07
1,GIRO2464133,GIRO2464133,None,OGM735,CATITA VITORIA DE OG,F,F,None,None,EFC383,...,None,None,None,37634-ABCZ,PO,None,14617590.0,2464133,ABCZ,2023-03-07
2,GIRO3311494,GIRO3311494,None,MBLO12,BAILARINA,F,F,None,None,RRP6097,...,None,None,None,37634-ABCZ,PO,None,17695912.0,3311494,ABCZ,2023-03-07
3,GIRO3275092,GIRO3275092,None,MBLO18,FANTASIA,F,F,None,None,GAV291,...,None,None,None,37634-ABCZ,PO,None,17697290.0,3275092,ABCZ,2023-03-07
4,GIRO2020509,GIRO2020509,None,IVAR5160,PANTOJA VILLEFORT,F,F,None,None,CAL6557,...,None,None,None,37634-ABCZ,PO,None,15053366.0,2020509,ABCZ,2023-03-07
5,GIRO2131978,GIRO2131978,None,APKK18,IBERICA FARDO,F,F,None,None,MUT697,...,None,None,None,27111-ABCZ,PO,None,13672891.0,2131978,ABCZ,2023-03-07
6,GIRO1989512,GIRO1989512,None,GBEZ7,FIV 5035,F,F,None,None,GAV291,...,None,None,None,5678-GIRO,PO,None,14810603.0,1989512,ABCZ,2023-03-07
7,GIRO2075525,GIRO2075525,None,PVBG1140,GARFIEIRA,F,F,None,None,103136,...,None,None,None,5407-GIRO,PO,None,13662231.0,2075525,ABCZ,2023-03-07
8,GIRO2075526,GIRO2075526,None,PVBG1091,GL CERINA CRISTAL,F,F,None,None,GAV291,...,None,None,None,5407-GIRO,PO,None,13594612.0,2075526,ABCZ,2023-03-07
9,GIRO2075524,GIRO2075524,None,PVBG855,FORTUNA DA CRISTAL,F,F,None,None,103136,...,None,None,None,5407-GIRO,PO,None,13215608.0,2075524,ABCZ,2023-03-07
